In [50]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import random
import json
import Augmentor


In [51]:
#Loading Variables
WIDTH = 1600
HEIGHT = 900
ORANGE_BACKGROUND = [12, 255, 255]
ROOT_DIR = os.path.abspath("./")
IMGS_DIR = os.path.join(ROOT_DIR, "Img")
AUG_INPUT_DIR = os.path.join(ROOT_DIR, IMGS_DIR, "AugmentorInput")
f = open('./Img/annotations.json', 'r')
annotations = json.load(f)
orange_signs_list = os.listdir('./Img/RawOrangeSignImgs/')
coco_imgs_list = os.listdir('./coco_dataset/images/chosenImages')
daily_templates_list = os.listdir('./Img/Templates/Daily')
frequent_templates_list = os.listdir('./Img/Templates/Frequent')
uncommom_templates_list = os.listdir('./Img/Templates/Uncommon') 
templates_list = [(daily_templates_list, frequent_templates_list, 
                   uncommom_templates_list)]
coco_imgs_list = coco_imgs_list[:2000]




In [52]:
def aplyAugmentationToImage():
    """
    Applies augmentation to a list of images
    """
    p = Augmentor.Pipeline(AUG_INPUT_DIR)
    
    

In [53]:
def applyErosion(mask):
    '''
    Apply Erosion to the mask.
    -Erosion makes the mask smaller and removes noise.
    :param mask: Mask to be eroded.
    :return: Eroded mask.   
    ''' 
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    return mask

def getContours(mask):
    '''
    Uses OpenCV to find the contours of the mask.
    To be a valid contour, it must have an area of at least 600px².
    :param mask: Mask to be used to find contours.
    :return: List of valid contours. 
    '''
    valid_contours_list = []
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        area = cv2.contourArea(c)
        if area < 600:
            continue
        valid_contours_list.append(c)
    return valid_contours_list

def createMask(img):
    '''
    Creates a mask from the image.
    In this project the image is a sign with a orange background.
    :param img: Image to be used to create the mask.
    :return: Mask created from the image.
    '''
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  #Convert img from BGR to HSV
    # print("HSV Img Shape: ", hsv_img.shape)

    lower_limit = np.array(ORANGE_BACKGROUND) #Background HSV Color to create mask
    upper_limit = np.array(ORANGE_BACKGROUND) #Background HSV Color to create mask

    mask = cv2.inRange(hsv_img, lower_limit, upper_limit) #Select Orange area
    mask_inv = cv2.bitwise_not(mask)   #Invert mask (deselect orang area) and select sign area       
    mask_inv = applyErosion(mask_inv)       #Erode the mask to remove noise
    #print("Mask-Inv Shape:" + mask_inv.shape)  #should be (h, w, 3)
    
    return mask_inv

def randomResize(img):
    '''
    Resizes the image to a random size.
    :param img: Image to be resized.
    :return: Resized image.
    '''
    random_scale = random.uniform(0.3, 1.1)
    width, height = int(img.shape[1] * random_scale), int(img.shape[0] * random_scale)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
    return img


def createTemplate(orange_signs_list):
    '''
    Creates a template for every orange sign from input list. 
    And saves it in a dictionary.
    :param orange_signs_list: List of orange signs loaded from ./Img/RawOriginSign.
    :return template_dict: Dictionary with the templates for each orange sign.
    
    template_dict = { sign_type1: [(sing_number, sign_rgba, contours_list)]
                      sign_type2: [(sing_number, sign_rgba, contours_list), 
                                   (sing_number, sign_rgba, contours_list)]
                      ...}),
    '''
    templates_dict = {}
    for name in orange_signs_list:
        name = name[:-4:]               # remove .jpg
        sign_type = name[:-4:]          # select the sign type
        sign_number = name[-4::]        # select the number of this sign type
        img = cv2.imread(f'./Img/RawOrangeSignImgs/{sign_type + sign_number}.jpg')
        img = randomResize(img)
        # cv2.imshow("Img", img)
        # print('Img Shape:', img.shape)     

        mask = createMask(img)
        contours_list = getContours(mask) #Needed to the annotations

        ## Using mask to select orange area and cut it from img, creating a transparent sign template
        sign = cv2.bitwise_and(img, img, mask= mask)
        ## Convert to RGBA (RGB with Alpha Channel)
        sign_rgba = cv2.cvtColor(sign, cv2.COLOR_BGR2BGRA)
        sign_rgba[:,:,3] = mask     #Add mask to alpha channel

        ## Add new tuple to dictionary if it doesn't exist
        if not templates_dict.get(sign_type):  
            templates_dict[sign_type] = [(sign_number, sign_rgba, contours_list)]
        ## The tuples contains the sign number (_XXX), sign image -> RGBA: (XXX, XXX, 4)) 
        # and contours list [shape: (n, 1, 2)]]                                 
        else: 
            templates_dict[sign_type].append((sign_number, sign_rgba, contours_list))
    
    return templates_dict
        
templates_dict = createTemplate(orange_signs_list)

In [54]:
SAVE = True
ANNOTATIONS = True
DRAW_CONTOURS = False

def addBlur(img):
    '''
    Adds a blur to the image.
    :param img: Image to be blurred.
    :return: Blurred image.
    '''
    kernel = np.ones((5,5), np.float32)/25  #Blur kernel with a size of 5x5 and a factor of 1/25 (default)
    img = cv2.filter2D(img, -1, kernel)     #Blur image using kernel
    return img

def checkIntervals(a, b, c, d):
    '''
    Checks if a < b and c < d.
    :param a: First interval start.
    :param b: First interval end.
    :param c: Second interval start.
    :param d: Second interval end.
    :return: True if b-a and d-c are greater than 0, False otherwise.
    '''
    if a < b and c < d:
        return True
    else:
        return False

def chooseOffset(i, temp_w, temp_h):
    '''
    Depending on the quadrant we are drawing the template, we choose a different random offset.
    :param i: Quadrant we are drawing the template.
    :param temp_w: Template width.
    :param temp_h: Template height.
    :return: Random offset for the specific quadrant.
    '''
    if i == 0:
        valid_intervals = checkIntervals(temp_w, WIDTH/2-temp_w, temp_h, HEIGHT/2-temp_h)
        # Change these to change the position of first sign
        if valid_intervals:
            x_offset, y_offset= np.random.randint(temp_w, WIDTH/2-temp_w), np.random.randint(temp_h, HEIGHT/2-temp_h)     
        else:
            x_offset, y_offset= np.random.randint(5, temp_w), np.random.randint(5, temp_h)     
    elif i == 1:
        valid_intervals = checkIntervals(WIDTH/2+temp_w, WIDTH - temp_w, temp_h, HEIGHT/2-temp_h)
        # Change these to change the position of second sign
        if valid_intervals:
            x_offset, y_offset= np.random.randint(WIDTH/2 + temp_w, WIDTH - temp_w), np.random.randint(temp_h, HEIGHT/2-temp_h)     
        else:
            x_offset, y_offset= np.random.randint(WIDTH/2, WIDTH/2 + temp_w), np.random.randint(10, temp_h)     
    elif i == 2:
        valid_intervals = checkIntervals(temp_w, WIDTH/2 - temp_w, HEIGHT/2+temp_h, HEIGHT - temp_h)
        # Change these to change the position of third sign
        if valid_intervals:
            x_offset, y_offset= np.random.randint(temp_w, WIDTH/2 - temp_w), np.random.randint(HEIGHT/2 + temp_h, HEIGHT - temp_h)
        else:
            x_offset, y_offset= np.random.randint(10, temp_w), np.random.randint(HEIGHT/2 + 10, HEIGHT/2 + temp_h)
    else:
        valid_intervals = checkIntervals(WIDTH/2+temp_w, WIDTH - temp_w, HEIGHT/2+temp_h, HEIGHT - temp_h)
        # Change these to change the position of fourth sign
        if valid_intervals:
            x_offset, y_offset= np.random.randint(WIDTH/2+temp_w, WIDTH - temp_h), np.random.randint(HEIGHT/2+temp_h, HEIGHT-temp_h)     
        else:
            x_offset, y_offset= np.random.randint(WIDTH/2+10, WIDTH/2 + temp_w), np.random.randint(HEIGHT/2+10, HEIGHT/2 + temp_h)

    return x_offset, y_offset


def addOffsetToContour(x_offset, y_offset, contours): 
    '''
    Add offset to contours according to the position that we paste the sign template.
    :param x_offset: X offset.
    :param y_offset: Y offset.
    :param contours: Contours to be offset.
    :return: Offsetted contours.
    '''
    for contour in contours:
        new_contour = contour + (x_offset, y_offset)
    return new_contour


def createPointsFromContours(contours):
    '''
    Create list of points_x and points_y from contours, to be used for create JSON object for annotations.
    :param contours: Contours that gonna be used to extract values for points_x and points_y.
    :return: Two lists: points_x, points_y
    '''
    points_x = []
    points_y = []
    for contour in contours:
        for point in contour:
            points_x.append(point[0])
            points_y.append(point[1])
    assert len(points_x) == len(points_y), "Error: points_x and points_y have different length"    
    return points_x, points_y

def pasteTemplateIntoCocoImage(coco_img, templates_list, templates_dict):
    '''
    Uses arbitrary image `coco_img` and pastes beteen 1 and 4 templates of random signs into it.
    :param coco_img: Image to be used to paste templates.
    :param templates_list: List of templates with 3 sublists, each of then containnig signs of one these categories:
        Daily: Signs that are the most for the day user.
        Frequent: Signs that everybody have seen at least once.
        Uncommom: Signs that probably aren't used often and fewer people know about them.
    :param templates_dict: Dictionary with the templates and contours for each sign.
    :return coco_img, countErrors, regions: Image with templates pasted, number of errors when pasting and regions to be used for annotations.
    '''
    countErrors = 0
    numberOfSigns = np.random.randint(1, 5) # Random number of signs to be placed in the image [1, 4]
    
    ## Unpack templates_list
    daily_templates_list, frequent_templates_list, uncommom_templates_list = templates_list[0] 
    
    chosen_list = [] # List to save 1 to 4 signs templates to paste in the image
    regions = [] # List to save object of regions to be used for annotations
      

    while numberOfSigns > 0:
        ## Choose random sign from daily, frequent or uncommom list, with weights of 0.5:0.3:0.2
        chosen_frequency = random.choices(
            [daily_templates_list, frequent_templates_list, uncommom_templates_list], 
            weights=[0.7, 0.25, 0.05], k=1)[0]
        ## Choose random sign from chosen list
        chosen_sign = random.choices(chosen_frequency, k=1)[0] 
        chosen_sign = chosen_sign[:-4:] # remove .jpg
        chosen_list.append(chosen_sign)
        numberOfSigns -= 1

    
    for i, sign in enumerate(chosen_list):
        sign_number = int(sign[-3::]) # select the number of this sign type
        sign_type = sign[:-4:] # select the sign type

        template_tuple = templates_dict[sign_type][sign_number] # get the template tuple for this sign
        template = template_tuple[1] # get the template image
        temp_h, temp_w, _ = template.shape # template height and width
        # print(template.shape)
        
        ## Choose offset for the chosen sign template
        x_offset, y_offset = chooseOffset(i, temp_w, temp_h) 
        
        ## Using the offset to drag the image down and to the right    
        y1, y2 = y_offset, y_offset + template.shape[0]
        x1, x2 = x_offset, x_offset + template.shape[1]

        alpha_s = template[:, :, 3] / 255.0 # get the alpha channel of the template
        alpha_l = 1.0 - alpha_s  # alpha_l is the alpha of the background of the template
        
        for c in range(0, 3):
            try:
                ## paste the template on the image
                coco_img[y1:y2, x1:x2, c] = (alpha_s * template[:, :, c] +
                                    alpha_l * coco_img[y1:y2, x1:x2, c])   
                ## Add offset to contours
                template_countour = addOffsetToContour(x_offset, y_offset, template_tuple[2])  

                if DRAW_CONTOURS:
                    ## Draw contours on the images to check if positions are correct
                    cv2.drawContours(coco_img, [template_countour], -1, (0, 255, 0), 3) 
                
                ## Create points_x and points_y from contours for annotations    
                points_x, points_y = createPointsFromContours(template_countour)    

                annot_obj = {           # Create object to convert for JSON annotations
                    "shape_attributes": {
                        "name": "polygon",
                        "all_points_x": points_x,
                        "all_points_y": points_y
                    },
                    "region_attributes": 
                        {
                            "class": sign_type
                        }
                }
                regions.append(annot_obj) # Add JSON object to regions list
            except:
                countErrors+= 1 # if the template is bigger than the image, the program will crash
                #print(f"Error: template {sign_type}_{sign_number} is bigger than the image")
                continue  
        
    return coco_img, countErrors, regions

def createSampleImages(templates_dict, coco_imgs_list, templates_list, blurry):
    '''
    Creates artificial images with templates of signs pasted in them.
    :param templates_dict: Dictionary with the templates and contours for each sign.
    :param coco_imgs_list: List of images to be used to paste templates.
    :param templates_list: List of templates with 3 sublists, each of then containnig signs of one these categories:
        Daily: Signs that are the most for the day user.
        Frequent: Signs that everybody have seen at least once.
        Uncommom: Signs that probably aren't used often and fewer people know about them.
    :param blurry: Boolean variable to determine if the images will be blurred or not.
    :return: Object with annotations for all images generated.
    '''
    img_list = []
    annot_obj = {}     # Create empty object to save annotations
    totalErrors = 0    # Count Errors when trying to paste the template into the image

    for name in coco_imgs_list:
        name = name[:-4:]       # remove .jpg
        coco_img = cv2.imread(f'./coco_dataset/images/chosenImages/{name}.jpg')
        ## Eliminate images with height greater than width (portraits)   
        if(coco_img.shape[0] > coco_img.shape[1]):    
            continue

        ## Resize the image to 1600x900    
        coco_img = cv2.resize(coco_img, (WIDTH, HEIGHT), interpolation = cv2.INTER_LINEAR) 
        
        ## Paste templates into the image
        coco_img, errors, regions_list = pasteTemplateIntoCocoImage(coco_img, templates_list, templates_dict) 
        totalErrors += errors   #Add errors to total errors      
        
        if blurry:
            coco_img = addBlur(coco_img) #Add blur to the image
            
        if SAVE:
            ## Save the image to the disk
            cv2.imwrite(f'./Img/ArtificialSamples/{name}.jpg', coco_img) 
            ## Get the size of the image
            img_size = str(os.stat(f'./Img/ArtificialSamples/{name}.jpg').st_size) 
            if ANNOTATIONS:
        
                img_annotation = { 
                    name+'.jpg'+img_size: 
                    {
                        "file_name": name+'.jpg',
                        "size": img_size,
                        "regions": regions_list,
                        "file_attributes": {}
                    }
                }

                annot_obj.update(img_annotation) # Add image annotation to the object
        
                 
        img_list.append(coco_img)
    print(f'Total errors: {totalErrors/3}')
    return annot_obj

    # cv2.imshow('Img', img_list[1])
    # cv2.waitKey(0)

new_annotations = createSampleImages(templates_dict, coco_imgs_list, templates_list, blurry=True)

Total errors: 85.0


In [55]:
annotations.update(new_annotations)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('new_annotations.json', 'w') as f:
    json.dump(annotations, f, cls=NpEncoder)
